In [10]:
import json

def refine_cluster_mapping(mapping_file='cluster_mapping.json', keep_prefixes=None, output_file=None):
    """
    cluster_mapping.json 파일의 내용을 정제하여 KEEP_PREFIXES에 있는 필드만 추출하고
    각 클러스터별로 정리해서 보여주는 함수
    
    Args:
        mapping_file: cluster_mapping.json 파일 경로
        keep_prefixes: 유지할 필드 접두사 튜플 (None이면 기본값 사용)
        output_file: 정제된 결과를 저장할 파일 경로 (None이면 저장하지 않음)
    
    Returns:
        정제된 클러스터 매핑 딕셔너리
    """
    # keep_prefixes 설정
    if keep_prefixes is None:
        keep_prefixes = ('processName', 'eventName', 'syscall', 'args', 'executable', 'returnValue')
    
    # cluster_mapping.json 파일 읽기
    with open(mapping_file, 'r', encoding='utf-8') as f:
        mapping = json.load(f)
    
    # 정제된 결과 저장용
    refined_mapping = {
        'total_clusters': mapping.get('total_clusters', 0),
        'total_samples': mapping.get('total_samples', 0),
        'keep_prefixes': list(keep_prefixes),
        'clusters': {}
    }
    
    # 각 클러스터별로 정제
    for cluster_id, cluster_info in mapping.get('clusters', {}).items():
        refined_cluster = {
            'cluster_id': int(cluster_id) if cluster_id != '-1' else -1,
            'count': cluster_info.get('count', 0),
            'data': []
        }
        
        # 각 데이터 항목에서 KEEP_PREFIXES에 해당하는 필드만 추출
        for item in cluster_info.get('data', []):
            original_data = item.get('data', {})
            refined_data = {
                'index': item.get('index'),
                'filtered_data': {}
            }
            
            # 원본 데이터가 딕셔너리인 경우
            if isinstance(original_data, dict):
                for key, value in original_data.items():
                    # keep_prefixes에 해당하는 키만 추출
                    if any(key.startswith(prefix) for prefix in keep_prefixes):
                        refined_data['filtered_data'][key] = value
            else:
                # 딕셔너리가 아닌 경우 그대로 저장
                refined_data['filtered_data'] = original_data
            
            refined_cluster['data'].append(refined_data)
        
        # 클러스터별 통계 정보 추가
        if refined_cluster['data']:
            # 첫 번째 샘플 데이터
            first_sample = refined_cluster['data'][0].get('filtered_data', {})
            refined_cluster['sample_data'] = first_sample
            
            # 주요 필드별 통계
            process_names = []
            event_names = []
            syscalls = []
            
            for item in refined_cluster['data']:
                filtered = item.get('filtered_data', {})
                if isinstance(filtered, dict):
                    if 'processName' in filtered:
                        process_names.append(filtered['processName'])
                    if 'eventName' in filtered:
                        event_names.append(filtered['eventName'])
                    if 'syscall' in filtered:
                        syscalls.append(filtered['syscall'])
            
            refined_cluster['statistics'] = {
                'unique_processes': len(set(process_names)) if process_names else 0,
                'unique_events': len(set(event_names)) if event_names else 0,
                'unique_syscalls': len(set(syscalls)) if syscalls else 0,
                'most_common_process': max(set(process_names), key=process_names.count) if process_names else None,
                'most_common_event': max(set(event_names), key=event_names.count) if event_names else None,
                'most_common_syscall': max(set(syscalls), key=syscalls.count) if syscalls else None,
            }
        
        refined_mapping['clusters'][cluster_id] = refined_cluster
    
    # 결과 출력
    print("=" * 80)
    print("정제된 클러스터 매핑 정보")
    print("=" * 80)
    print(f"총 클러스터 수: {refined_mapping['total_clusters']}")
    print(f"총 샘플 수: {refined_mapping['total_samples']}")
    print(f"유지된 필드 접두사: {refined_mapping['keep_prefixes']}")
    print()
    
    # 클러스터별 요약 출력 (크기 순으로 정렬)
    sorted_clusters = sorted(
        refined_mapping['clusters'].items(),
        key=lambda x: x[1]['count'],
        reverse=True
    )
    
    print("클러스터별 요약:")
    print("-" * 80)
    for cluster_id, cluster_info in sorted_clusters:
        stats = cluster_info.get('statistics', {})
        print(f"\n[클러스터 {cluster_id}] 데이터 개수: {cluster_info['count']}")
        if stats:
            print(f"  - 고유 프로세스 수: {stats['unique_processes']}")
            print(f"  - 고유 이벤트 수: {stats['unique_events']}")
            print(f"  - 고유 시스템콜 수: {stats['unique_syscalls']}")
            if stats['most_common_process']:
                print(f"  - 가장 흔한 프로세스: {stats['most_common_process']}")
            if stats['most_common_event']:
                print(f"  - 가장 흔한 이벤트: {stats['most_common_event']}")
            if stats['most_common_syscall']:
                print(f"  - 가장 흔한 시스템콜: {stats['most_common_syscall']}")
        
        # 샘플 데이터 출력
        sample = cluster_info.get('sample_data', {})
        if sample:
            print(f"  - 샘플 데이터: {sample}")
    
    # 파일로 저장 (옵션)
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(refined_mapping, f, ensure_ascii=False, indent=2)
        print(f"\n정제된 결과가 {output_file}에 저장되었습니다.")
    
    return refined_mapping

# 사용 예시
# refined = refine_cluster_mapping(
#     mapping_file='cluster_mapping.json',
#     output_file='cluster_mapping_refined.json'  # 선택사항: 저장하지 않으려면 None
# )

In [11]:
import pandas as pd
import json

def refined_json_to_dataframe(json_file='cluster_mapping_refined.json'):
    """
    cluster_mapping_refined.json 파일을 읽어서 데이터프레임으로 변환하는 함수
    
    Args:
        json_file: cluster_mapping_refined.json 파일 경로
    
    Returns:
        pandas DataFrame: 각 행이 하나의 샘플 데이터를 나타냄
                         컬럼: cluster_id, index, 그리고 filtered_data의 모든 필드
    """
    # JSON 파일 읽기
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 모든 샘플 데이터를 저장할 리스트
    rows = []
    
    # 각 클러스터별로 데이터 추출
    for cluster_id, cluster_info in data.get('clusters', {}).items():
        cluster_id_int = cluster_info.get('cluster_id', int(cluster_id) if cluster_id != '-1' else -1)
        
        # 각 데이터 항목을 행으로 변환
        for item in cluster_info.get('data', []):
            row = {
                'cluster_id': cluster_id_int,
                'index': item.get('index')
            }
            
            # filtered_data의 모든 필드를 행에 추가
            filtered_data = item.get('filtered_data', {})
            if isinstance(filtered_data, dict):
                for key, value in filtered_data.items():
                    # 중첩된 객체(예: args, executable)는 JSON 문자열로 변환
                    if isinstance(value, (dict, list)):
                        row[key] = json.dumps(value, ensure_ascii=False)
                    else:
                        row[key] = value
            else:
                row['filtered_data'] = filtered_data
            
            rows.append(row)
    
    # DataFrame 생성
    df = pd.DataFrame(rows)
    
    # 컬럼 순서 정리 (cluster_id, index를 앞에 배치)
    if not df.empty:
        cols = ['cluster_id', 'index']
        other_cols = [c for c in df.columns if c not in cols]
        df = df[cols + other_cols]
    
    return df

# 사용 예시
# df = refined_json_to_dataframe('cluster_mapping_refined.json')
# print(df.head())
# print(f"\n데이터프레임 shape: {df.shape}")
# print(f"\n컬럼: {df.columns.tolist()}")

In [13]:
# 사용 예시
refined = refine_cluster_mapping(
     mapping_file='cluster_mapping.json',
     output_file ='cluster_mapping_refined.json')  # 선택사항: 저장하지 않으려면 None

정제된 클러스터 매핑 정보
총 클러스터 수: 8
총 샘플 수: 721
유지된 필드 접두사: ['processName', 'eventName', 'syscall', 'args', 'executable', 'returnValue']

클러스터별 요약:
--------------------------------------------------------------------------------

[클러스터 5] 데이터 개수: 258
  - 고유 프로세스 수: 1
  - 고유 이벤트 수: 1
  - 고유 시스템콜 수: 1
  - 가장 흔한 프로세스: init
  - 가장 흔한 이벤트: setresgid
  - 가장 흔한 시스템콜: setresgid
  - 샘플 데이터: {'processName': 'init', 'executable': {'path': ''}, 'eventName': 'setresgid', 'argsNum': 3, 'returnValue': 0, 'syscall': 'setresgid', 'args': [{'name': 'rgid', 'type': 'int32', 'value': -1}, {'name': 'egid', 'type': 'int32', 'value': 1000}, {'name': 'sgid', 'type': 'int32', 'value': -1}]}

[클러스터 6] 데이터 개수: 258
  - 고유 프로세스 수: 1
  - 고유 이벤트 수: 1
  - 고유 시스템콜 수: 1
  - 가장 흔한 프로세스: init
  - 가장 흔한 이벤트: setresuid
  - 가장 흔한 시스템콜: setresuid
  - 샘플 데이터: {'processName': 'init', 'executable': {'path': ''}, 'eventName': 'setresuid', 'argsNum': 3, 'returnValue': 0, 'syscall': 'setresuid', 'args': [{'name': 'ruid', 'type': 'int32', '

In [14]:
df = refined_json_to_dataframe('cluster_mapping_refined.json')
print(df.head())
print(f"\n데이터프레임 shape: {df.shape}")
print(f"\n컬럼: {df.columns.tolist()}")

   cluster_id  index processName    executable                eventName  \
0           0      0       initd  {"path": ""}  security_socket_connect   
1           0      5       initd  {"path": ""}  security_socket_connect   
2           0     10       initd  {"path": ""}  security_socket_connect   
3           0     15       initd  {"path": ""}  security_socket_connect   
4           0     18       initd  {"path": ""}  security_socket_connect   

   argsNum  returnValue  syscall  \
0        3            0  connect   
1        3            0  connect   
2        3            0  connect   
3        3            0  connect   
4        3            0  connect   

                                                args  
0  [{"name": "sockfd", "type": "int32", "value": ...  
1  [{"name": "sockfd", "type": "int32", "value": ...  
2  [{"name": "sockfd", "type": "int32", "value": ...  
3  [{"name": "sockfd", "type": "int32", "value": ...  
4  [{"name": "sockfd", "type": "int32", "value": ...  

데

In [15]:
import numpy as np

def match_sequence_with_clusters(original_sequence, cluster_labels):
    """
    원본 시퀀스와 클러스터링 결과를 매칭하여 각 항목에 클러스터 ID를 추가하는 함수
    
    Args:
        original_sequence: 원본 데이터 시퀀스 (리스트 또는 배열)
                          - 리스트: [data1, data2, data3, ...]
                          - DataLoader의 dataset.original_data
                          - 또는 다른 순서가 보장된 시퀀스
        cluster_labels: 클러스터링 결과 레이블 (numpy array 또는 리스트)
                       - 길이는 original_sequence와 동일해야 함
                       - 각 값은 해당 인덱스의 클러스터 ID
    
    Returns:
        list: 각 항목이 {'data': 원본데이터, 'cluster_id': 클러스터ID, 'index': 인덱스} 형태인 리스트
    """
    # 입력 검증
    if len(original_sequence) != len(cluster_labels):
        raise ValueError(
            f"원본 시퀀스 길이({len(original_sequence)})와 클러스터 레이블 길이({len(cluster_labels)})가 일치하지 않습니다."
        )
    
    # cluster_labels를 numpy array로 변환 (리스트인 경우 대비)
    if not isinstance(cluster_labels, np.ndarray):
        cluster_labels = np.array(cluster_labels)
    
    # 매칭된 결과 리스트 생성
    matched_sequence = []
    
    for idx, (data, cluster_id) in enumerate(zip(original_sequence, cluster_labels)):
        matched_item = {
            'index': idx,
            'cluster_id': int(cluster_id),
            'data': data
        }
        matched_sequence.append(matched_item)
    
    return matched_sequence

# 사용 예시
# # 예시 1: 리스트 형태의 원본 데이터
# original_data = [{'processName': 'init'}, {'processName': 'bash'}, {'processName': 'ls'}]
# labels = np.array([0, 1, 0])
# matched = match_sequence_with_clusters(original_data, labels)
# print("매칭 결과:")
# for item in matched:
#     print(f"  인덱스 {item['index']}: 클러스터 {item['cluster_id']}, 데이터: {item['data']}")
# 
# # 예시 2: DataLoader의 original_data 사용
# # matched = match_sequence_with_clusters(train_normal_loader.dataset.original_data, labels)

In [16]:
# match_sequence_with_clusters 함수 실행 구조
import os

def load_original_sequence_from_file(data_file='data/tr_normal_tracee.json'):
    """
    원본 JSON Lines 파일에서 원본 시퀀스를 로드하는 함수
    
    Args:
        data_file: 원본 데이터 파일 경로
    
    Returns:
        list: 원본 데이터 시퀀스
    """
    original_sequence = []
    
    if not os.path.exists(data_file):
        raise FileNotFoundError(f"데이터 파일을 찾을 수 없습니다: {data_file}")
    
    with open(data_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    json_obj = json.loads(line)
                    original_sequence.append(json_obj)
                except json.JSONDecodeError:
                    continue
    
    return original_sequence

def load_cluster_labels_from_mapping(mapping_file='cluster_mapping.json', convert_noise_to_cluster=True):
    """
    cluster_mapping.json에서 클러스터 레이블을 재구성하는 함수
    
    Args:
        mapping_file: cluster_mapping.json 파일 경로
        convert_noise_to_cluster: True이면 노이즈(-1)를 별도의 클러스터 ID로 변환
    
    Returns:
        numpy array: 인덱스 순서대로 정렬된 클러스터 레이블
    """
    with open(mapping_file, 'r', encoding='utf-8') as f:
        mapping = json.load(f)
    
    # 인덱스와 클러스터 ID 매핑 딕셔너리 생성
    index_to_cluster = {}
    cluster_ids_set = set()
    
    for cluster_id, cluster_info in mapping.get('clusters', {}).items():
        cluster_id_int = int(cluster_id) if cluster_id != '-1' else -1
        
        for item in cluster_info.get('data', []):
            idx = item.get('index')
            if idx is not None:
                index_to_cluster[idx] = cluster_id_int
                if cluster_id_int != -1:
                    cluster_ids_set.add(cluster_id_int)
    
    # 최대 인덱스 찾기
    if not index_to_cluster:
        raise ValueError("클러스터 매핑에 데이터가 없습니다.")
    
    max_index = max(index_to_cluster.keys())
    
    # 노이즈를 별도 클러스터 ID로 변환
    if convert_noise_to_cluster:
        # 기존 클러스터 ID 중 최대값 찾기
        if cluster_ids_set:
            max_cluster_id = max(cluster_ids_set)
            next_cluster_id = max_cluster_id + 1
        else:
            next_cluster_id = 0
        
        # 노이즈 포인트를 개별 클러스터로 변환 (각각 다른 ID 부여)
        noise_indices = [idx for idx, cid in index_to_cluster.items() if cid == -1]
        noise_count = len(noise_indices)
        
        if noise_count > 0:
            # 각 노이즈 포인트에 고유한 클러스터 ID 부여
            for i, idx in enumerate(noise_indices):
                index_to_cluster[idx] = next_cluster_id + i
            
            if noise_count == 1:
                print(f"   노이즈 포인트(-1) 1개를 클러스터 ID {next_cluster_id}로 변환했습니다.")
            else:
                print(f"   노이즈 포인트(-1) {noise_count}개를 클러스터 ID {next_cluster_id}~{next_cluster_id + noise_count - 1}로 변환했습니다.")
        else:
            print("   노이즈 포인트가 없습니다.")
    
    # 인덱스 순서대로 클러스터 레이블 배열 생성
    cluster_labels = np.zeros(max_index + 1, dtype=int)
    
    for idx, cluster_id in index_to_cluster.items():
        cluster_labels[idx] = cluster_id
    
    return cluster_labels


In [17]:
# matched_sequence를 파일로 저장
def save_matched_sequence(matched_sequence, output_file='matched_sequence.json'):
    """
    matched_sequence를 JSON 파일로 저장하는 함수
    
    Args:
        matched_sequence: match_sequence_with_clusters 함수의 결과
        output_file: 저장할 파일 경로
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(matched_sequence, f, ensure_ascii=False, indent=2)
    
    print(f"matched_sequence가 {output_file}에 저장되었습니다.")
    print(f"총 {len(matched_sequence)}개의 항목이 저장되었습니다.")

# matched_sequence가 존재하는 경우 저장
if 'matched_sequence' in globals() and matched_sequence:
    save_matched_sequence(matched_sequence, 'matched_sequence.json')
else:
    print("matched_sequence 변수가 없습니다. 먼저 셀 5를 실행해주세요.")

matched_sequence 변수가 없습니다. 먼저 셀 5를 실행해주세요.


In [18]:

# 실행 구조
print("=" * 80)
print("원본 시퀀스와 클러스터 매칭 실행")
print("=" * 80)

# 1. 원본 데이터 시퀀스 로드
print("\n1. 원본 데이터 시퀀스 로드 중...")
original_sequence = load_original_sequence_from_file('data/tr_normal_tracee.json')
print(f"   로드된 데이터 개수: {len(original_sequence)}")

# 2. 클러스터 레이블 로드
print("\n2. 클러스터 레이블 로드 중...")
cluster_labels = load_cluster_labels_from_mapping('cluster_mapping.json')
print(f"   클러스터 레이블 개수: {len(cluster_labels)}")
print(f"   고유 클러스터 수: {len(set(cluster_labels))}")

# 3. 길이 확인 및 조정 (필요한 경우)
print("\n3. 데이터 길이 확인...")
if len(original_sequence) != len(cluster_labels):
    min_len = min(len(original_sequence), len(cluster_labels))
    print(f"   경고: 길이가 다릅니다. 원본: {len(original_sequence)}, 레이블: {len(cluster_labels)}")
    print(f"   최소 길이({min_len})로 조정합니다.")
    original_sequence = original_sequence[:min_len]
    cluster_labels = cluster_labels[:min_len]

# 4. match_sequence_with_clusters 함수 실행
print("\n4. 시퀀스와 클러스터 매칭 중...")
matched_sequence = match_sequence_with_clusters(original_sequence, cluster_labels)
print(f"   매칭 완료! 총 {len(matched_sequence)}개 항목")

# 5. 결과 확인
print("\n5. 매칭 결과 샘플 (처음 5개):")
print("-" * 80)
for i, item in enumerate(matched_sequence[:5]):
    data = item['data']
    process_name = data.get('processName', 'N/A')
    event_name = data.get('eventName', 'N/A')
    print(f"  인덱스 {item['index']:3d}: 클러스터 {item['cluster_id']:2d} | "
          f"프로세스: {process_name:15s} | 이벤트: {event_name}")

# 6. 클러스터별 통계
print("\n6. 클러스터별 데이터 개수:")
print("-" * 80)
from collections import Counter
cluster_counts = Counter([item['cluster_id'] for item in matched_sequence])
for cluster_id, count in sorted(cluster_counts.items()):
    print(f"  클러스터 {cluster_id:2d}: {count:4d}개")

print("\n" + "=" * 80)
print("매칭 완료!")
print("=" * 80)
print(f"\nmatched_sequence 변수에 결과가 저장되었습니다.")
print(f"총 {len(matched_sequence)}개의 매칭된 항목이 있습니다.") 

원본 시퀀스와 클러스터 매칭 실행

1. 원본 데이터 시퀀스 로드 중...
   로드된 데이터 개수: 721

2. 클러스터 레이블 로드 중...
   노이즈 포인트(-1) 23개를 클러스터 ID 7~29로 변환했습니다.
   클러스터 레이블 개수: 721
   고유 클러스터 수: 30

3. 데이터 길이 확인...

4. 시퀀스와 클러스터 매칭 중...
   매칭 완료! 총 721개 항목

5. 매칭 결과 샘플 (처음 5개):
--------------------------------------------------------------------------------
  인덱스   0: 클러스터  0 | 프로세스: initd           | 이벤트: security_socket_connect
  인덱스   1: 클러스터  7 | 프로세스: tracee          | 이벤트: security_socket_connect
  인덱스   2: 클러스터  1 | 프로세스: docker-desktop- | 이벤트: security_socket_connect
  인덱스   3: 클러스터  1 | 프로세스: docker-desktop- | 이벤트: security_socket_connect
  인덱스   4: 클러스터  1 | 프로세스: docker-desktop- | 이벤트: security_socket_connect

6. 클러스터별 데이터 개수:
--------------------------------------------------------------------------------
  클러스터  0:   62개
  클러스터  1:   16개
  클러스터  2:   53개
  클러스터  3:   15개
  클러스터  4:   36개
  클러스터  5:  258개
  클러스터  6:  258개
  클러스터  7:    1개
  클러스터  8:    1개
  클러스터  9:    1개
  클러스터 10:    1개
  클러스터 11:    1개
  클러

In [ ]:
matched_sequence 

[{'index': 0,
  'cluster_id': 0,
  'data': {'timestamp': 1770361145686246159,
   'threadStartTime': 1770357810525966063,
   'processorId': 9,
   'processId': 21,
   'cgroupId': 1,
   'threadId': 365,
   'parentProcessId': 1,
   'hostProcessId': 1632,
   'hostThreadId': 2164,
   'hostParentProcessId': 1612,
   'userId': 0,
   'mountNamespace': 4026532390,
   'pidNamespace': 4026532391,
   'processName': 'initd',
   'executable': {'path': ''},
   'hostName': 'docker-desktop',
   'containerId': '',
   'container': {},
   'kubernetes': {},
   'eventId': '736',
   'eventName': 'security_socket_connect',
   'matchedPolicies': [''],
   'argsNum': 3,
   'returnValue': 0,
   'syscall': 'connect',
   'stackAddresses': None,
   'contextFlags': {'containerStarted': False, 'isCompat': False},
   'threadEntityId': 1496401534,
   'processEntityId': 2522573051,
   'parentEntityId': 463719362,
   'args': [{'name': 'sockfd', 'type': 'int32', 'value': 178},
    {'name': 'type', 'type': 'int32', 'value': 

In [ ]:
matched_sequence 

[{'index': 0,
  'cluster_id': 0,
  'data': {'timestamp': 1770361145686246159,
   'threadStartTime': 1770357810525966063,
   'processorId': 9,
   'processId': 21,
   'cgroupId': 1,
   'threadId': 365,
   'parentProcessId': 1,
   'hostProcessId': 1632,
   'hostThreadId': 2164,
   'hostParentProcessId': 1612,
   'userId': 0,
   'mountNamespace': 4026532390,
   'pidNamespace': 4026532391,
   'processName': 'initd',
   'executable': {'path': ''},
   'hostName': 'docker-desktop',
   'containerId': '',
   'container': {},
   'kubernetes': {},
   'eventId': '736',
   'eventName': 'security_socket_connect',
   'matchedPolicies': [''],
   'argsNum': 3,
   'returnValue': 0,
   'syscall': 'connect',
   'stackAddresses': None,
   'contextFlags': {'containerStarted': False, 'isCompat': False},
   'threadEntityId': 1496401534,
   'processEntityId': 2522573051,
   'parentEntityId': 463719362,
   'args': [{'name': 'sockfd', 'type': 'int32', 'value': 178},
    {'name': 'type', 'type': 'int32', 'value': 